In [ ]:

!pip install boto3 sagemaker


In [1]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
import boto3
import datetime
import time
from time import strftime,gmtime
import json
import os
import urllib
import sys
import io

boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
region = boto_session.region_name
sns_client = boto3.client('sns',region_name=region)

bucket="sm-ball-tracking-output-blobs"
prefix = 'async-inference'

print(region)
print(prefix)

ca-central-1
async-inference


### Delete Role

In [24]:
import boto3
import json
from botocore.exceptions import ClientError

# Specify the role name
role_name = 'SageMaker-Role'

# Create an IAM client
iam_client = boto3.client('iam')

try:
    response = iam_client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    # Detach policies
    for policy in attached_policies:
        policy_arn = policy['PolicyArn']
        iam_client.detach_role_policy(RoleName=role_name, PolicyArn=policy_arn)
        print(f"Detached policy: {policy_arn}")

# Delete inline policies
    iam_client.delete_role(RoleName=role_name)
except Exception as e:
    print(e)

Detached policy: arn:aws:iam::aws:policy/AmazonSNSFullAccess
Detached policy: arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
Detached policy: arn:aws:iam::aws:policy/AmazonS3FullAccess


### Create Role

In [25]:
import boto3
import json
from botocore.exceptions import ClientError

# Specify the role name
role_name = 'SageMaker-Role'

# Managed policies for SageMaker
managed_policy_arns = [
    'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
    'arn:aws:iam::aws:policy/AmazonS3FullAccess',
    'arn:aws:iam::aws:policy/AmazonSNSFullAccess'
]

# Create an IAM client
iam_client = boto3.client('iam')
    
# Create the role
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "sagemaker.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )
    print("SageMaker role created successfully:", create_role_response['Role']['Arn'])
    
    # Attach managed policies to the role
    for policy_arn in managed_policy_arns:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"Attached policy {policy_arn} to the role.")

except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role with the same name already exists.")
    else:
        print("Error creating SageMaker role:", e)


SageMaker role created successfully: arn:aws:iam::800241512715:role/SageMaker-Role
Attached policy arn:aws:iam::aws:policy/AmazonSageMakerFullAccess to the role.
Attached policy arn:aws:iam::aws:policy/AmazonS3FullAccess to the role.
Attached policy arn:aws:iam::aws:policy/AmazonSNSFullAccess to the role.


In [26]:
role_arn = create_role_response['Role']['Arn']

In [2]:
role_arn="arn:aws:iam::800241512715:role/SageMaker-Role"

### Create Failure and Success Notification Topic

In [41]:
response = sns_client.create_topic(Name="Async-ML-ErrorTopic")
error_topic= response['TopicArn']
print(error_topic)

arn:aws:sns:ca-central-1:800241512715:Async-ML-ErrorTopic


In [42]:
response = sns_client.create_topic(Name="Async-ML-SuccessTopic")
success_topic = response['TopicArn']
print(success_topic)

arn:aws:sns:ca-central-1:800241512715:Async-ML-SuccessTopic


In [43]:
response = sns_client.list_topics()
topics = response["Topics"]
print(topics)

[{'TopicArn': 'arn:aws:sns:ca-central-1:800241512715:Async-ML-ErrorTopic'}, {'TopicArn': 'arn:aws:sns:ca-central-1:800241512715:Async-ML-SuccessTopic'}, {'TopicArn': 'arn:aws:sns:ca-central-1:800241512715:Test-VoD-v120-NotificationSnsTopicB941FD22-lz6dg1qRRgHh'}, {'TopicArn': 'arn:aws:sns:ca-central-1:800241512715:aws-controltower-SecurityNotifications'}, {'TopicArn': 'arn:aws:sns:ca-central-1:800241512715:test-new-videos'}]


In [44]:


email_id = 'mfahadm8@gmail.com'
email_sub_1 = sns_client.subscribe(
    TopicArn=success_topic,
    Protocol='email',
    Endpoint=email_id)

email_sub_2 = sns_client.subscribe(
    TopicArn=error_topic,
    Protocol='email',
    Endpoint=email_id)


## Async Endpoint Creation Step By Step

#### Create/Update Model Artifacts in S3

In [6]:
!rm -f model.tar.gz

In [7]:
!tar -czvf model.tar.gz yolov7.pt code >> /dev/null 2>&1
!aws s3 cp model.tar.gz s3://sm-ball-tracking-inputs/models/ >> /dev/null 2>&1

###  Model Config Creation In Sagemaker

In [45]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.c5.xlarge'
pytorch_inference_image_uri = retrieve('pytorch',
                                       region,
                                       version='1.12',
                                       py_version='py38',
                                       instance_type = deploy_instance_type,
                                       accelerator_type=None,
                                       image_scope='inference')
print(pytorch_inference_image_uri)

763104351884.dkr.ecr.ca-central-1.amazonaws.com/pytorch-inference:1.12-cpu-py38


In [46]:
try:
    model_name = 'ball-tracking-yolo7'
    create_model_response = sm_client.delete_model( ModelName = model_name)
except:
    print("Model not found!")

In [47]:
container = pytorch_inference_image_uri
model_artifact="s3://sm-ball-tracking-inputs/models/model.tar.gz"
model_name = 'ball-tracking-yolo7'
print(container)
print(model_name)

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role_arn,
    PrimaryContainer = {
        'Image': container,
        'ModelDataUrl': model_artifact,
        'Mode': 'SingleModel',
        'Environment': {
            'SAGEMAKER_CONTAINER_LOG_LEVEL':'20',
            'SAGEMAKER_PROGRAM':'inference.py',
            'SAGEMAKER_REGION':region,
            'SAGEMAKER_SUBMIT_DIRECTORY':'/opt/ml/model/code',
            'TS_MAX_REQUEST_SIZE': '100000000', #default max request size is 6 Mb for torchserve, need to update it to support the 70 mb input payload
            'TS_MAX_RESPONSE_SIZE': '100000000',
            'TS_DEFAULT_RESPONSE_TIMEOUT': '1000'
        }
    },    
)

763104351884.dkr.ecr.ca-central-1.amazonaws.com/pytorch-inference:1.12-cpu-py38
ball-tracking-yolo7


### Endpoint Config Creation For Sagemaker

In [48]:
try:
    endpoint_config_name = "BallTrackingV7AsyncEndpointConfig"
    create_endpoint_config_response = sm_client.delete_endpoint_config(
    EndpointConfigName=endpoint_config_name
    )
except:
    print("Endpoint Configuratoin not found!")

Endpoint Configuratoin not found!


In [49]:
print(model_name)
endpoint_config_name = "BallTrackingV7AsyncEndpointConfig"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.c5.xlarge",
            "InitialInstanceCount": 1
        }
    ],
    AsyncInferenceConfig={
        "OutputConfig": {
            "S3OutputPath": f"s3://{bucket}/{prefix}/output",
            #  Optionally specify Amazon SNS topics
            "NotificationConfig": {
              "SuccessTopic": success_topic,
              "ErrorTopic": error_topic,
            },
            "S3FailurePath": f"s3://{bucket}/{prefix}/failure",
        },
        "ClientConfig": {
            "MaxConcurrentInvocationsPerInstance": 2
        }
    }
)
print(f"Created EndpointConfig: {create_endpoint_config_response['EndpointConfigArn']}")

ball-tracking-yolo7


Created EndpointConfig: arn:aws:sagemaker:ca-central-1:800241512715:endpoint-config/balltrackingv7asyncendpointconfig


### Endpoint Creation

deleting existing endpoint

In [13]:
try:
    endpoint_name = "ball-tracking-v7"
    sm_client.delete_endpoint(EndpointName=endpoint_name)
except Exception as e:
    print(e)

In [14]:
endpoint_name = "ball-tracking-v7"
endpoint_config_name= "BallTrackingV7AsyncEndpointConfig"
create_endpoint_response = sm_client.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
print(f"Creating Endpoint: {create_endpoint_response['EndpointArn']}")

Creating Endpoint: arn:aws:sagemaker:ca-central-1:800241512715:endpoint/ball-tracking-v7


In [55]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

Waiting for endpoint to create...


WaiterError: Waiter EndpointInService failed: Waiter encountered a terminal failure state: Matched expected service error code: ValidationException

## Testing

In [17]:
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
import uuid
import os

LABELS_BUCKET="lightsketch-models-188775091215"
VIDEO_BUCKET="lightsketch-models-188775091215"
input_video_uri='s3://lightsketch-models-188775091215/models/20200616_VB_trim.mp4'
inference_id=str(uuid.uuid4())
endpoint_name="ball-tracking-v7"
predictor=Predictor(endpoint_name=endpoint_name,sagemaker_session=sm_session,serializer=JSONSerializer())
async_predictor = AsyncPredictor(predictor=predictor)
s3_input_path_without_prefix = input_video_uri[len("s3://"):]
input_bucket_name, input_key = s3_input_path_without_prefix.split('/', 1)
input_base_file = os.path.basename(input_key)
input_base_filename= os.path.splitext(input_base_file)[0]

input_data = {
    'input_location': input_video_uri,
    'output_label_location':  "s3://"+LABELS_BUCKET+"/"+inference_id+"/"+input_base_filename+".csv",
    'output_video_location':  "s3://"+VIDEO_BUCKET+"/"+inference_id+"/"+input_base_file
}


input_s3_uri=f"s3://{bucket}/{prefix}/input/{inference_id}.json"
# Call the predict method to send the input data to the endpoint asynchronously
response = async_predictor.predict_async(data=input_data,input_path=input_s3_uri,inference_id=inference_id)


In [20]:

result=response.get_result().decode('utf-8')
error_message = json.loads(result)["Error"]
print(error_message)

Traceback (most recent call last):
  File "/opt/ml/model/code/inference.py", line 75, in transform_fn
    ouput_path= detect(local_filename,model,output_label_location,output_video_location)
  File "/opt/ml/model/code/inference.py", line 215, in detect
    online_targets = tracker.update(np.array(dets), [old_img_w, old_img_h], (img.shape[3], img.shape[2]))
  File "/opt/ml/model/code/tracker/byte_tracker.py", line 168, in update
    if output_results.shape[1] == 5:
IndexError: tuple index out of range

